In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# import findspark
# findspark.init()
# if sc: sc.stop()
from pyspark.sql import SparkSession, Row
from pyspark import SparkContext
from pyspark.sql.types import StructType, StructField, ArrayType, StringType

sc = SparkContext()
spark = SparkSession(sc)

ModuleNotFoundError: No module named 'pyspark'

In [10]:
schema = StructType([StructField('temps',ArrayType(StructType( \
              [StructField('current',ArrayType(StructType \
                  ([StructField('temp',StringType(),True)]),True),True), \
               StructField('hist',ArrayType(StructType \
                  ([StructField('temp',StringType(),True)]),True), \
         True)]),True),True),
         StructField('uuid',StringType(),True)])

In [29]:
A = [Row(uuid='100001', temps=[Row(current=[Row(temp=None)], hist=[Row(temp='27')])]),
     Row(uuid='100002', temps=[Row(current=[Row(temp=None)], hist=[Row(temp='33')])])]
B = [Row(uuid='100001', temps=[Row(current=[Row(temp='30')], hist=[Row(temp=None)])]),
     Row(uuid='100002', temps=[Row(current=[Row(temp='25')], hist=[Row(temp=None)])])]

dfA = sc.parallelize(A).toDF(schema)
dfA.registerTempTable('dfA')
dfB = sc.parallelize(B).toDF(schema)
dfB.registerTempTable('dfB')
dfA.collect()
dfB.collect()
dfA.show()
dfB.show()

[Row(temps=[Row(current=[Row(temp=None)], hist=[Row(temp='27')])], uuid='100001'),
 Row(temps=[Row(current=[Row(temp=None)], hist=[Row(temp='33')])], uuid='100002')]

[Row(temps=[Row(current=[Row(temp='30')], hist=[Row(temp=None)])], uuid='100001'),
 Row(temps=[Row(current=[Row(temp='25')], hist=[Row(temp=None)])], uuid='100002')]

+----------------+------+
|           temps|  uuid|
+----------------+------+
|[[[[]], [[27]]]]|100001|
|[[[[]], [[33]]]]|100002|
+----------------+------+

+----------------+------+
|           temps|  uuid|
+----------------+------+
|[[[[30]], [[]]]]|100001|
|[[[[25]], [[]]]]|100002|
+----------------+------+



In [267]:
# dfA.printSchema()

dfC = dfA \
    .select(F.explode('temps').alias('tmp')) \
    .select('tmp.*') \
    .show()

+-------+----+
|current|hist|
+-------+----+
|[[101]]|[[]]|
+-------+----+



In [34]:
# dfD = dfA.join(dfB, 'uuid').show()
# dfD.show()

dfA.show()
dfB.show()



+----------------+------+
|           temps|  uuid|
+----------------+------+
|[[[[]], [[27]]]]|100001|
|[[[[]], [[33]]]]|100002|
+----------------+------+

+----------------+------+
|           temps|  uuid|
+----------------+------+
|[[[[30]], [[]]]]|100001|
|[[[[25]], [[]]]]|100002|
+----------------+------+



In [30]:
sql = '''
SELECT
    dfA.uuid,
    (dfB.temps.current, dfA.temps.hist) as temps
FROM 
    dfA
LEFT OUTER JOIN
    dfB
ON dfA.uuid = dfB.uuid
'''

dfC = spark.sql(sql)
dfC.collect()

[Row(uuid='100002', temps=Row(current=[[Row(temp='25')]], hist=[[Row(temp='33')]])),
 Row(uuid='100001', temps=Row(current=[[Row(temp='30')]], hist=[[Row(temp='27')]]))]